In [1]:
import numpy as np
import seaborn as sns
from time import time

import tensorflow as tf
from tensorflow import keras
from keras import layers, models
from keras.regularizers import L1L2
import keras.backend as K

import os
import gc
from pathlib import Path

tf.config.experimental.set_visible_devices([], 'GPU')

Using TensorFlow backend.


In [2]:
from tensorflow.keras.losses import Huber

In [3]:
init_state = np.array([
    [0, 0, 0],
    [0, 0, 0],
    [0, 0, 0]
])

In [4]:
class Game:
    def __init__(self, state, FIRST=1):
        self.state = state
        self.empty = self.make_empty(state)
        self.first_player = FIRST
        
    def make_empty(self, state):
        emp = []
        for i in range(3):
            for j in range(3):
                if state[i][j] == 0:
                    emp.append(3*i + j)
        return emp
    
    def is_lose(self):
        a = self.next_opp()
        
        for i in range(3):
            if self.state[i][0] == self.state[i][1] == self.state[i][2] == a:
                return True
            elif self.state[0][i] == self.state[1][i] == self.state[2][i] == a:
                return True
        if self.state[0][0] == self.state[1][1] == self.state[2][2] == a:
            return True
        if self.state[0][2] == self.state[1][1] == self.state[2][0] == a:
            return True
        return 0
    
    def is_draw(self):
        a = self.next_opp()
        if self.is_lose():
            return 0
        if np.all(self.state):
            return 1
        else:
            return 0
        
    def is_done(self):
        if self.is_lose() or self.is_draw():
            return 1
        else:
            return 0
        
        
    def update(self, target):
        state = self.state.copy()
        x, y = target//3, target%3
        a = self.next_opp()
        state[x][y] = a
        return Game(state)
    
    
    def next_opp(self):
        a = b = 0
        for i in range(len(self.state)):
            for j in range(len(self.state)):
                if self.state[i][j] == self.first_player:
                    a += 1
                elif self.state[i][j] != 0:
                    b += 1
                    
        if a == b:
            return self.first_player
        else:
            return 2 + min(0, 1-self.first_player)

In [5]:
class Random:
    def action(self, game):
        return np.random.choice(game.empty)

In [6]:
def playout(game):
    if game.is_lose():
        return -1

    if game.is_draw():
        return 0

    return -playout(game.update(np.random.choice(game.empty)))


def action(game):
    n_steps=50
    values = [0] * 9
    for i in range(9):
        if i in game.empty:
            for _ in range(n_steps):
                g = game.update(i)
                values[i] += playout(g)
            
    for i in range(9):
        values[i] /= n_steps
        
    return values

In [ ]:
DN_FILTERS = 128  # 컨볼루션 레이어 커널 수(오리지널 256）
DN_RESIDUAL_NUM = 16  # 레지듀얼 블록 수(오리지널 19)
DN_INPUT_SHAPE = (3, 3, 2)  # 입력 셰이프
DN_OUTPUT_SIZE = 9  # 행동 수(배치 수(3*3))
    
def residual_block():
    def f(x):
        sc = x
        x = layers.Conv2D(DN_FILTERS, 3, padding='same', use_bias=False,
              kernel_initializer='he_normal', kernel_regularizer=L1L2(l2=0.0005))(x)
        x = layers.BatchNormalization()(x)
        x = layers.Activation('relu')(x)
        x = layers.Conv2D(DN_FILTERS, 3, padding='same', use_bias=False,
              kernel_initializer='he_normal', kernel_regularizer=L1L2(l2=0.0005))(x)
        x = layers.BatchNormalization()(x)
        x = layers.Add()([x, sc])
        x = layers.Activation('relu')(x)
        return x

    return f
    
def dual_network():
    # 모델 생성이 완료된 경우 처리하지 않음
#     if os.path.exists('./model/best.h5'):
#         return

    # 입력 레이어
    input = layers.Input(shape=DN_INPUT_SHAPE)

    # 컨볼루션 레이어
    x = layers.Conv2D(DN_FILTERS, 3, padding='same', use_bias=False,
              kernel_initializer='he_normal', kernel_regularizer=L1L2(l2=0.0005))(input)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)

    # 레지듀얼 블록 x 16
    for i in range(DN_RESIDUAL_NUM):
        x = residual_block()(x)

    # 풀링 레이어
    x = layers.GlobalAveragePooling2D()(x)

    # policy 출력
    p = layers.Dense(DN_OUTPUT_SIZE, kernel_regularizer=L1L2(l2=0.0005),
              activation='softmax', name='pi')(x)

    # value 출력
#     v = layers.Dense(1, kernel_regularizer=L1L2(l2=0.0005))(x)
#     v = layers.Activation('tanh', name='v')(v)

    # 모델 생성
    model = models.Model(inputs=input, outputs=p)

    model.compile(optimizer = 'adam',
                 loss = 'categorical_crossentropy')

    return model

class CNN:
    def __init__(self):
        K.clear_session()
        self.model = dual_network()
        
    def action(self, game):
        status = game.next_opp()
        target = np.reshape(game.state, (1, 1, 3, 3)).astype('float')
        target = np.where(target==status, 1., np.where(target==0, 0, 2.))
        res = self.model.predict(target)[0]
        a = np.argmax(res)
        while a not in game.empty:
            res[a] = 0
            a = np.argmax(res)
        
        return a

    def train(self, n=200):
        X = []
        y = []
        for _ in range(n):
            print(_)
            for i in [1, 2]:
                game = Game(init_state)

                while 1:
                    status = game.next_opp()
                    state = self.make_state(game)
                    values = action(game)
                    a = np.argmax(values)
                    
                    X.append(state)
                    temp_y = [0] * 9
                    temp_y[a] = np.max(values)
                    y.append(temp_y)

                    game = game.update(a)
                    
                    if game.is_done():
                        break
                        
        es = keras.callbacks.EarlyStopping(patience=20, restore_best_weights=True)

        X = np.reshape(X, (len(X), 1, 3, 3))
        y = np.reshape(y, (len(y), 9))   
        print(len(X))
        self.model.fit(X, y, epochs=1000, verbose=1, 
                       callbacks=[es], validation_split=0.2,
                      batch_size = 256)
        
    def make_state(self, game):
        status = game.next_opp()
        opp = 3 - status
        a = game.state
        a1 = np.where(a==status, 1, 0)
        a2 = np.where(a==opp, 1, 0)
        res = np.array([a1, a2])
        
        return res


In [ ]:
dd = CNN()
gc.collect()

In [ ]:
dd.train(100)
gc.collect()

In [ ]:
# for _ in range(100):
#     print(_)
#     dd.train()

In [ ]:
a = init_state.copy()
a = np.reshape(a, (1, 1, 3, 3)).astype('float')
a = np.where(a==1, 1., np.where(a==0, 0, -1.))

dd.model.predict(a)

In [ ]:
def play(game, m1, m2):
    global score
    while 1:
        a1 = m1.action(game)
        game = game.update(a1)
#         print(game.state)
        if game.is_lose():
            score[0] += 1
#             print(game.state)
            return 
        elif game.is_draw():
            score[2] += 1
#             print(game.state)
            return 

        a2 = m2.action(game)
        game = game.update(a2)
#         print(game.state)
        if game.is_lose():
            score[1] += 1
#             print(game.state)
            return 
        elif game.is_draw():
            score[2] += 1
#             print(game.state)
            return 
        

In [ ]:
game = Game(init_state)
m1 = Random()
# d = DQN(1)

In [ ]:
dd.model.predict(np.reshape(init_state, (1, 1, 3, 3))).astype('f')[0]

In [ ]:
%%time
score = [0, 0, 0]
for _ in range(100):
#     print(_)
    play(game, m1, dd)
print(score)
gc.collect()

score = [0, 0, 0]
for _ in range(100):
#     print(_)
    play(game, dd, m1)
print(score)

In [ ]:
# sns.barplot(x = [1, 2], y = score[:2])

In [ ]:
# softmax + cross_entropy
# v1: 3vs7 // 3vs4
# v2: 